In [1]:
import numpy as np
import pandas as pd

In [2]:
trade_df = pd.read_stata("tradeflows.dta")
trade_df.head()

,year,iso_o,iso_d,contig,comlang_off,distw,pop_o,gdp_o,gdpcap_o,iso2_o,...,rta,comleg,comcur,acp_to_eu,gsp,eu_to_acp,gsp_rec,flow,validmirror,family
0,1999,ABW,AGO,0,0,9587.315563,NaN,1722.905029,NaN,AW,...,0,1,0,0,0,0,0,0.0,0,
1,2000,ABW,AGO,0,0,9587.315563,NaN,1858.659180,NaN,AW,...,0,1,0,0,0,0,0,0.0,1,
2,2001,ABW,AGO,0,0,9587.315563,NaN,1898.882690,NaN,AW,...,0,1,0,0,0,0,0,0.0,1,
3,2002,ABW,AGO,0,0,9587.315563,NaN,1911.173218,NaN,AW,...,0,1,0,0,0,0,0,0.0,1,
4,2003,ABW,AGO,0,0,9587.315563,0.099,NaN,NaN,AW,...,0,1,0,0,0,0,0,0.0,1,


In [3]:
trade_df = trade_df.loc[:,['year','iso_o','iso_d','gdp_o','gdp_d','flow', 'distw']]
trade_df.head()

,year,iso_o,iso_d,gdp_o,gdp_d,flow,distw
0,1999,ABW,AGO,1722.905029,6154.479980,0.0,9587.315563
1,2000,ABW,AGO,1858.659180,9129.179688,0.0,9587.315563
2,2001,ABW,AGO,1898.882690,8936.023438,0.0,9587.315563
3,2002,ABW,AGO,1911.173218,11431.738281,0.0,9587.315563
4,2003,ABW,AGO,NaN,13956.267578,0.0,9587.315563


In [4]:
polity_scores_df = pd.read_excel("polity.xls")
polity_scores_df.head()

,cyear,ccode,scode,country,year,flag,fragment,democ,autoc,polity,...,interim,bmonth,bday,byear,bprec,post,change,d4,sf,regtrans
0,7001800,700,AFG,Afghanistan,1800,0,NaN,1,7,-6,...,NaN,1.0,1.0,1800.0,1.0,-6.0,88.0,1.0,NaN,NaN
1,7001801,700,AFG,Afghanistan,1801,0,NaN,1,7,-6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7001802,700,AFG,Afghanistan,1802,0,NaN,1,7,-6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7001803,700,AFG,Afghanistan,1803,0,NaN,1,7,-6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7001804,700,AFG,Afghanistan,1804,0,NaN,1,7,-6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
trade_df[trade_df['iso_o'] == "RUS"].head()

,year,iso_o,iso_d,gdp_o,gdp_d,flow,distw
938582,2000,RUS,ABW,259708.50000,1858.659180,0.000000,10523.265397
938583,2001,RUS,ABW,306602.68750,1898.882690,0.090015,10523.265397
938584,2002,RUS,ABW,345470.50000,1911.173218,0.000000,10523.265397
938585,2003,RUS,ABW,431487.03125,NaN,0.000000,10523.265397
938586,2004,RUS,ABW,591742.43750,NaN,64.816180,10523.265397


In [6]:
# Russia?
polity_scores_df[polity_scores_df['scode'] == "RUS"].head()

,cyear,ccode,scode,country,year,flag,fragment,democ,autoc,polity,...,interim,bmonth,bday,byear,bprec,post,change,d4,sf,regtrans
13252,3651800,365,RUS,Russia,1800,0,NaN,0,10,-10,...,NaN,1.0,1.0,1800.0,1.0,-10.0,88.0,1.0,NaN,NaN
13253,3651801,365,RUS,Russia,1801,0,NaN,0,10,-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13254,3651802,365,RUS,Russia,1802,0,NaN,0,10,-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13255,3651803,365,RUS,Russia,1803,0,NaN,0,10,-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13256,3651804,365,RUS,Russia,1804,0,NaN,0,10,-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
same_countries = sorted(list(set(trade_df['iso_o'].unique()) & set(polity_scores_df['scode'].unique())))
print(same_countries)
print("Polity - Same =", set(polity_scores_df['scode'].unique()).difference(set(same_countries)))

['AFG', 'AGO', 'ALB', 'ARG', 'ARM', 'AUS', 'AUT', 'AZE', 'BDI', 'BEL', 'BEN', 'BFA', 'BGD', 'BGR', 'BHR', 'BIH', 'BLR', 'BOL', 'BRA', 'BTN', 'BWA', 'CAF', 'CAN', 'CHE', 'CHL', 'CHN', 'CMR', 'COL', 'COM', 'CPV', 'CRI', 'CUB', 'CYP', 'CZE', 'DEU', 'DJI', 'DNK', 'DOM', 'DZA', 'ECU', 'EGY', 'ERI', 'ESP', 'EST', 'ETH', 'FIN', 'FJI', 'FRA', 'GAB', 'GBR', 'GEO', 'GHA', 'GMB', 'GNB', 'GNQ', 'GRC', 'GTM', 'GUY', 'HND', 'HRV', 'HTI', 'HUN', 'IDN', 'IND', 'IRL', 'IRN', 'IRQ', 'ISR', 'ITA', 'JAM', 'JOR', 'JPN', 'KAZ', 'KEN', 'KGZ', 'KHM', 'KOR', 'KWT', 'LBN', 'LBR', 'LBY', 'LKA', 'LSO', 'LTU', 'LUX', 'LVA', 'MAR', 'MDA', 'MDG', 'MEX', 'MKD', 'MLI', 'MNG', 'MOZ', 'MRT', 'MUS', 'MWI', 'MYS', 'NAM', 'NER', 'NGA', 'NIC', 'NLD', 'NOR', 'NPL', 'NZL', 'OMN', 'PAK', 'PAN', 'PER', 'PHL', 'PNG', 'POL', 'PRT', 'PRY', 'QAT', 'RUS', 'RWA', 'SAU', 'SDN', 'SEN', 'SGP', 'SLB', 'SLE', 'SLV', 'SOM', 'SUR', 'SVN', 'SWE', 'SWZ', 'SYR', 'TCD', 'TGO', 'THA', 'TJK', 'TKM', 'TTO', 'TUN', 'TUR', 'TZA', 'UGA', 'UKR', 'URY'

In [8]:
polity_scores_df = polity_scores_df.loc[:,['scode', 'country', 'year', 'polity']]
polity_scores_df.head()

,scode,country,year,polity
0,AFG,Afghanistan,1800,-6
1,AFG,Afghanistan,1801,-6
2,AFG,Afghanistan,1802,-6
3,AFG,Afghanistan,1803,-6
4,AFG,Afghanistan,1804,-6


In [9]:
trade_df = trade_df[trade_df['iso_o'].isin(same_countries)]
# Create new column with trade link (iso_d + iso_d)
trade_df["tradelink"] = trade_df.apply(lambda row: row['iso_o'] + '-' + row['iso_d'], axis=1)
trade_df.reset_index(drop=True, inplace=True)

,year,iso_o,iso_d,gdp_o,gdp_d,flow,distw,tradelink
0,2000,AFG,ANT,NaN,NaN,0.0,13004.563789,AFG-ANT
1,2001,AFG,ANT,2461.638916,NaN,0.0,13004.563789,AFG-ANT
2,2002,AFG,ANT,4036.688721,NaN,0.0,13004.563789,AFG-ANT
3,2003,AFG,ANT,4584.684570,NaN,0.0,13004.563789,AFG-ANT
4,2004,AFG,ANT,5951.966797,NaN,0.0,13004.563789,AFG-ANT


In [10]:
# Remove all rows with iso_d not in same countries
trade_df = trade_df[trade_df['iso_d'].isin(same_countries)]
trade_df.reset_index(drop=True, inplace=True)

In [ ]:
# Remove all rows with scode not in same countries
polity_scores_df = trade_df[polity_scores['scode'].isin(same_countries)]
polity_scores_df.reset_index(drop=True, inplace=True)

In [11]:
trade_df.describe()

,year,gdp_o,gdp_d,flow,distw
count,763240.000000,6.962420e+05,7.152430e+05,763240.000000,763240.000000
mean,1981.542301,1.482592e+05,1.447229e+05,177.146578,7467.645906
std,16.808470,6.792301e+05,6.702051e+05,2395.453869,4358.156507
min,1948.000000,2.520352e+01,2.794697e+01,0.000000,114.637288
25%,1968.000000,2.073000e+03,2.093392e+03,0.000000,3994.317629
50%,1984.000000,8.482883e+03,8.254573e+03,0.220000,6964.352921
75%,1996.000000,5.730673e+04,5.393397e+04,9.007058,10439.926170
max,2006.000000,1.320182e+07,1.320182e+07,348420.600000,19650.127278


In [13]:
from collections import defaultdict

In [ ]:
# Append Polity scores for iso_o column to trade_df
trade_df["polity_origin"] = None
missing_origin_values = defaultdict(list)
for i in range(len(trade_df)):
    try:
        origin = trade_df['iso_o'][i]
        year = trade_df['year'][i]
        polity = polity_scores_df[polity_scores_df['scode'] == origin]
        polity = polity[polity['year'] == year].reset_index(drop=True)
        score = polity['polity'][0]
        trade_df.loc[i, ["polity_origin"]] = score
    except:
        missing_origin_values[origin].append(year)
        trade_df.drop(trade_df.index[i])
trade_df.head(20)

In [ ]:
# Append Polity scores for iso_d column to trade_df
trade_df["polity_dest"] = None
missing_dest_values = defaultdict(list)
for i in range(len(trade_df)):
    try:
        origin = trade_df['iso_d'][i]
        year = trade_df['year'][i]
        polity = polity_scores_df[polity_scores_df['scode'] == origin]
        polity = polity[polity['year'] == year].reset_index(drop=True)
        trade_df.loc[i, ["polity_dest"]] = score
    except:
        missing_dest_values[origin].append(year)
        trade_df.drop(trade_df.index[i])
trade_df.head(20)

In [ ]:
# Append difference in scores column to trade_df
trade_df['polity_difference'] = abs(trade_df['polity_origin'] - trade_df['polity_dest'])
trade_df.head(20)

In [ ]:
trade_df['polity_dist'].unique()

In [ ]:
trade_df['log_gdp_o'] = np.log(trade_df.gdp_o)
trade_df['log_gdp_d'] = np.log(trade_df.gdp_d)
trade_df['log_flow'] = np.log(trade_df.flow)
trade_df['log_flow'] = trade_df['log_flow'].apply(lambda x: np.log(x) if not(math.isinf(x)) else 0)
trade_df['log_distw'] = np.log(trade_df.distw)
trade_df['log_polity_dist'] = trade_df['polity_dist'].apply(lambda x: np.log(int(x)) if (x != 0) else 0)
trade_df.head(10)

In [ ]:
trade_df = trade_df.set_index(['tradelink', 'year'])

In [ ]:
trade_df.to_excel('trade_polity.xlsx')